In [11]:
# Dependencies
import pandas as pd
import pymongo
import time
import datetime as dt
import geojson
import requests
import json

In [2]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.housing_db
collection = db.ny_map
df = pd.DataFrame(list(db.housing_summary.find()))
df

,_id,month,region_name,inventory,median_days_on_market,median_sale_price,price_drops
0,60952012d1bec65f2c9384ba,01,"Albany County, NY",7941.0,58.658333,218320.825000,361.0
1,60952012d1bec65f2c9384bb,01,"Albany, NY metro area",30142.0,61.941667,206972.016667,1167.0
2,60952012d1bec65f2c9384bc,01,"Allegany County, NY",1533.0,95.091667,65026.925000,34.0
3,60952012d1bec65f2c9384bd,01,"Amsterdam, NY metro area",1881.0,85.541667,106653.366667,63.0
4,60952012d1bec65f2c9384be,01,"Auburn, NY metro area",2553.0,74.375000,115040.266667,87.0
...,...,...,...,...,...,...,...
619,60952012d1bec65f2c938725,12,"Warren County, NY",2929.0,61.991667,232587.141667,61.0
620,60952012d1bec65f2c938726,12,"Washington County, NY",2285.0,81.308333,167634.916667,49.0
621,60952012d1bec65f2c938727,12,"Wayne County, NY",984.0,17.741667,155767.033333,31.0
622,60952012d1bec65f2c938728,12,"Westchester County, NY",30541.0,41.150000,601293.875000,660.0


In [3]:
df['region_name']= df['region_name'].str.split(',',expand=True)
# df['region_name']= df['region_name'].str.replace(" County", "")
df

,_id,month,region_name,inventory,median_days_on_market,median_sale_price,price_drops
0,60952012d1bec65f2c9384ba,01,Albany County,7941.0,58.658333,218320.825000,361.0
1,60952012d1bec65f2c9384bb,01,Albany,30142.0,61.941667,206972.016667,1167.0
2,60952012d1bec65f2c9384bc,01,Allegany County,1533.0,95.091667,65026.925000,34.0
3,60952012d1bec65f2c9384bd,01,Amsterdam,1881.0,85.541667,106653.366667,63.0
4,60952012d1bec65f2c9384be,01,Auburn,2553.0,74.375000,115040.266667,87.0
...,...,...,...,...,...,...,...
619,60952012d1bec65f2c938725,12,Warren County,2929.0,61.991667,232587.141667,61.0
620,60952012d1bec65f2c938726,12,Washington County,2285.0,81.308333,167634.916667,49.0
621,60952012d1bec65f2c938727,12,Wayne County,984.0,17.741667,155767.033333,31.0
622,60952012d1bec65f2c938728,12,Westchester County,30541.0,41.150000,601293.875000,660.0


In [8]:
df = df[df['region_name'].str.contains('(?!$)County')]
df

,month,region_name,inventory,median_days_on_market,median_sale_price,price_drops
0,01,Albany County,7941.0,58.658333,218320.825000,361.0
1,01,Allegany County,1533.0,95.091667,65026.925000,34.0
2,01,Bronx County,13722.0,86.358333,481706.591667,306.0
3,01,Cattaraugus County,3452.0,92.025000,92387.825000,110.0
4,01,Cayuga County,2553.0,74.375000,115040.266667,87.0
...,...,...,...,...,...,...
451,12,Warren County,2929.0,61.991667,232587.141667,61.0
452,12,Washington County,2285.0,81.308333,167634.916667,49.0
453,12,Wayne County,984.0,17.741667,155767.033333,31.0
454,12,Westchester County,30541.0,41.150000,601293.875000,660.0


In [5]:
df= df.groupby(['month','region_name']).agg({'inventory':'mean',
                         'median_days_on_market':'mean',
                         'median_sale_price':'mean',
                         'price_drops':'mean',
                        })
df

inventory  median_days_on_market  median_sale_price  \
month region_name                                                               
01    Albany County          7941.0              58.658333      218320.825000   
      Allegany County        1533.0              95.091667       65026.925000   
      Bronx County          13722.0              86.358333      481706.591667   
      Cattaraugus County     3452.0              92.025000       92387.825000   
      Cayuga County          2553.0              74.375000      115040.266667   
...                             ...                    ...                ...   
12    Warren County          2929.0              61.991667      232587.141667   
      Washington County      2285.0              81.308333      167634.916667   
      Wayne County            984.0              17.741667      155767.033333   
      Westchester County    30541.0              41.150000      601293.875000   
      Wyoming County          538.0              27.108333      133772.216667   

                          price_drops  
month region_name                      
01    Albany County             361.0  
      Allegany County            34.0  
      Bronx County              306.0  
      Cattaraugus County        110.0  
      Cayuga County              87.0  
...                               ...  
12    Warren County              61.0  
      Washington County          49.0  
      Wayne County               31.0  
      Westchester County        660.0  
      Wyoming County             27.0  

[456 rows x 4 columns]

In [9]:
df.reset_index(inplace = True)
data_dict = df.to_dict("records")

In [10]:
 # Insert dictionary into MongoDB as a document
    collection.insert_many(data_dict)